# TASK 4
Creazione di due sistemi di raccomandazione di ristoranti:
- **Content-based**: basato sulle caratteristiche del locale;
- **Collaborative**: sulla base delle preferenze degli utenti del sistema.

In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD

from libraries.dataset import Dataset
import libraries.data_handler as data_handler
import pandas as pd
import numpy as np
import ast

## Data retrieving
Ottenimento dei dati relativi alle recensioni, bilanciati sulla base delle stelle, al fine di ottenere lo stesso numero di review per ogni possibile valutazione (da 1 a 5).
In questo specifico caso, sono richiesti 20'000 campioni per ogni tipo di classe (per un totale di 100'000 campioni).

L'oggetto `review_data` contiene tre field relativi ai subdataset da utilizzare nel progetto:
- `train_data` = tupla contentente i dati ed i target per il training
- `val_data` = tupla contentente i dati ed i target per la validazione
- `test_data` = tupla contentente i dati ed i target per il testing

Alla prima esecuzione, i tre diversi subset sono memorizzati sottoforma di file csv, in modo da evitare la riesecuzione del codice di splitting dei dataset durante le successive esecuzioni.  

In [ ]:
review_data = Dataset('review', 'stars')

review_data.split(['user_id', 'business_id', 'text'], 'stars', n_samples=20_000)

Si è scelto di lavorare con un sottogruppo del dataset originale delle aziende, formato solo dai ristoranti collocati in Georgia, Florida e Texas. 


Per ottenere il subset, è stato inizialmente creato l'oggetto `business_data`, contenente tutte le informazioni relative alle attività all'interno dei tre stati.
In particolare, le features che compongono il dataset sono il *business id*, il *nome* dell'azienda, l'*indirizzo*, le *categorie* a cui appartiene l'attività, le *caratteristiche* (parcheggio, fascia di prezzo, ecc.) e il *voto medio* dell'attività in stelle.

In [ ]:
business_data = pd.DataFrame(data_handler.load_dataset('business'))
business_data = business_data[(business_data['state']=='GA') | (business_data['state'] == 'FL') | (business_data['state'] == 'TX')].reset_index()
business_data = business_data[['business_id','name','address', 'categories', 'attributes','stars']]

Sono stati poi selezionati solo i ristoranti ed il risultato è stato salvato nell'oggetto `restaurants_data`.

In [4]:
restaurants_data = business_data[business_data['categories'].str.contains('Restaurant.*')==True].reset_index()

Sono state definite delle funzioni per l'*estrazione* delle chiavi contenute all'interno di dizionari annidati nel dizionario della feature `attributes` e per la *conversione* di una stringa in dizionario.

In [5]:
def extract_keys(attr, key):
    if attr == None:
        return "{}"
    if key in attr:
        return attr.pop(key)

def str_to_dict(attr):
    if attr != None:
        return ast.literal_eval(attr)
    else:
        return ast.literal_eval("{}")   

In [6]:
def to_Dict(attribute):
    restaurants_data[attribute] = restaurants_data.apply(lambda x: str_to_dict(extract_keys(x['attributes'], attribute)), axis=1)

In [7]:
to_Dict('BusinessParking')
to_Dict('Ambience')
to_Dict('GoodForMeal')
to_Dict('Dietary')
to_Dict('Music')

Concatenazione delle serie relative alle features che formeranno il nuovo dataset e one-hot encoding di tali serie.

In [ ]:
df_attr = pd.concat([ restaurants_data['attributes'].apply(pd.Series), restaurants_data['BusinessParking'].apply(pd.Series),
                    restaurants_data['Ambience'].apply(pd.Series), restaurants_data['GoodForMeal'].apply(pd.Series), 
                    restaurants_data['Dietary'].apply(pd.Series)], axis=1)
df_attr_dummies = pd.get_dummies(df_attr)

In [ ]:
df_categories_dummies = pd.Series(restaurants_data['categories']).str.get_dummies(',')
df_categories_dummies

In [ ]:
result = restaurants_data[['name','stars']]
result

Il dataset finale è ottenuto dalla concatenazione di attributi, categorie e ristoranti.

Gli attributi e le categorie sono features trasformate con il one-hot encoding del passo precedente. Inoltre, le stelle sono convertite in interi per agevolarne l'utilizzo nell'addestramento dei modelli.

In [11]:
df_final = pd.concat([df_attr_dummies, df_categories_dummies, result], axis=1)
df_final.drop('Restaurants',inplace=True,axis=1)
df_final['stars'] = df_final['stars'].astype(int)

## Content Based Filtering

### KNN
Definizione di un Recommendation System che utilizza l'algoritmo KNN per l'ottenimento di ristoranti simili.

Il modello è stato addestrato sull'intero dataset escludendo l'ultimo sample, utilizzato per fase di testing. Il valore di k per il numero dei vicini è stato fissato a 5.

In [ ]:
knn = NearestNeighbors(n_neighbors=5)
knn.fit(df_final.iloc[:-1,:-2])

Visualizzazione del nome del ristorante scelto per la fase di testing.

In [15]:
test_sample = df_final.iloc[-1:,:-2]
print("Test set (Restaurant name): ", df_final['name'].values[-1])

Test set (Restaurant name):  Zora Grille


Calcolo dei vicini, ovvero i ristoranti da suggerire, rispetto al test sample.

In [17]:
distances, indices =  knn.kneighbors(test_sample)

final_table = pd.DataFrame(data={'index': indices[0], 'distance': distances[0]})
final_table.set_index('index')

,distance
index,
2390,3.316625
5135,3.464102
9057,3.464102
2605,3.605551
18103,3.605551


Visualizzazione dei ristoranti suggeriti sulla base delle caratteristiche del ristorante di test.

In [22]:
result = final_table.join(df_final,on='index')
result[['distance','index','name','stars']]

,distance,index,name,stars
0,3.316625,2390,Mezza,4
1,3.464102,5135,Royal Indian Cuisine,4
2,3.464102,9057,Ray's Rio Bravo,2
3,3.605551,2605,Nicola's Restaurant,4
4,3.605551,18103,Punjab Indian Restaurant,2


## Collaborative Based Filtering

## SVD

Definizione di un Recommendation System di tipo collaborative, che suggerisce ristoranti sulla base delle preferenze degli utenti nel sistema.

E' stata scelta la tecnica SVD, una particolare fattorizzazione di matrice basata sull'utilizzo di autovalori e autovettori.
Viene utilizzata una combinazione dei dataset delle *recensioni* e dei *ristoranti* al fine di ottenere una matrice. A partire da tale matrice si otterranno poi i ristoranti che risultano avere una correlazione particolarmente forte con il ristorante in input.

In [ ]:
restaurants = restaurants_data[['business_id', 'name', 'address']]
reviews = review_data.train_data[0]
reviews['stars'] = review_data.train_data[1]

combined_business_data = pd.merge(reviews, restaurants, on='business_id')
combined_business_data

Visualizzazione dei primi cinque ristoranti che hanno avuto il numero più alto di stelle da parte degli utenti.

In [24]:
combined_business_data.groupby('business_id')['stars'].count().sort_values(ascending=False).head()

business_id
bZiIIUcpgxh8mpKMDhdqbA    45
sPhPI3B6tvcJIULhICr-Pg    44
CxQ1m2iY4wQpXC64tSfWgQ    43
WkN8Z2Q8gbhjjkCt8cDVxg    43
MGzro82Fi4LYvc86acoONQ    42
Name: stars, dtype: int64

Si seleziona il ristorante che sarà l'input del recommendation system.

In [25]:
filtered = combined_business_data['business_id'] == 'bZiIIUcpgxh8mpKMDhdqbA'
print("Name: ", combined_business_data[filtered]['name'].unique())
print("Address:", combined_business_data[filtered]['address'].unique())

Name:  ['Hopdoddy Burger Bar']
Address: ['1400 S Congress Ave, Ste A190']


Calcolo della matrice le cui righe corrispondono agli utenti, le colonne ai ristoranti ed ogni cella rappresenta il numero di stelle che l'utente ha assegnato ad ogni ristorante.

In [ ]:
rating_crosstab = combined_business_data.pivot_table(values='stars', index='user_id', columns='name', fill_value=0)
rating_crosstab.head()

Fitting dell'SVD sulla matrice ottenuta.

In [27]:
SVD = TruncatedSVD(n_components=12, random_state=17)
result_matrix = SVD.fit_transform(rating_crosstab.values.T)

Calcolo della matrice dei coefficienti di correlazione a partire dalla matrice calcolata dall'SVD.

In [28]:
corr_matrix = np.corrcoef(result_matrix)

Ricerca dell'indice relativo al ristorante con il maggior numero di stelle.

In [29]:
restaurant_names = rating_crosstab.columns
restaurants_list = list(restaurant_names)

popular_rest = restaurants_list.index('Hopdoddy Burger Bar')
print("index of the popular restaurant: ", popular_rest) 

index of the popular restaurant:  2852


Selezione degli indici dei ristoranti fortemente correlati al ristorante in input.

In [30]:
corr_popular_rest = corr_matrix[popular_rest]
corrIndices = [i for i, x in enumerate((corr_popular_rest <= 1.0) & (corr_popular_rest > 0.9)) if x]
corrIndices

[948, 1809, 2257, 2397, 2852, 2973, 5114, 5563, 5811, 6230, 6653, 7112]

Visualizzazione dei 10 ristoranti suggeriti dal Recommendation System con il relativo coefficiente di correlazione.

In [31]:
recommendedRest = [(restaurant_names[i], corr_matrix[popular_rest, i]) for i in corrIndices]
recommendedRest.sort(key=lambda x: x[1], reverse=True)
recommendedRest.pop(0)
recommendedRest[:10]

[('Doghouse', 0.9999955245780303),
 ('Fogo De Chão Brazilian Steakhouse', 0.9999942452978231),
 ('Zaviya Grill', 0.9999122444240708),
 ('Tumble22', 0.9998342999940556),
 ('Independence Fine Foods', 0.9997540124544847),
 ('Social House Orlando', 0.9995505478815057),
 ("Rio's Brazilian Cafe", 0.9994173499828607),
 ('Sway', 0.9928870187377924),
 ('Bun Belly', 0.9915646397639292),
 ('The Funkadelic', 0.9519420018894759)]